In [1]:
import os

In [2]:
os.getcwd()
os.chdir("../")

In [3]:
os.getcwd()

'/home/ryefoxlime/MathTutor'

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataIngestion:
    docs: Path
    EMBED_MODEL: str
    model: str


In [12]:
from src.Math.constants import *
from src.Math.utils.common import read_yaml
from pathlib import Path

class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH):
        self.config=read_yaml(config_filepath)
    def get_data_ingestion_config(self) -> DataIngestion:
        config=self.config.data_ingestion

        data_ingestion_config = DataIngestion(
            docs=Path(config.data_folder),
            EMBED_MODEL=config.EMBED_MODEL,
            model=config.model,
        )
        return data_ingestion_config

In [22]:
from google import genai
from llama_index.readers.file import FlatReader
from llama_index.core.node_parser import SentenceSplitter
from src.Math import logger
class DataLoader:
    def __init__(self, config: DataIngestion):
        self.config = config
        self.splitter = SentenceSplitter(chunk_size=100, chunk_overlap=20)
        self.client = genai.Client()
        
    def load_and_chunk_pdf(self, filename):
                filename = self.config.docs / file.name
                logger.info(filename)
                docs = FlatReader().load_data(filename)
                texts = [d.text for d in docs if getattr(d, "text", None)]
                chunks = []
                for t in texts:
                    chunks.extend(self.splitter.split_text(t))
                return chunks
            
    def embed_texts(self, texts: list[str]):
        response = self.client.models.embed_content(
            model=self.config.EMBED_MODEL,
            contents=texts,
            config={
                
                "task_type":"RETRIEVAL_DOCUMENT",
            }
        )
        return [item.values for item in response.embeddings]

In [23]:
config = ConfigurationManager()
get_data_ingestion_config = config.get_data_ingestion_config()
Dataloader = DataLoader(config=get_data_ingestion_config)
if Path.is_dir(get_data_ingestion_config.docs):
    for file in get_data_ingestion_config.docs.iterdir():
        chuck = Dataloader.load_and_chunk_pdf(filename = file)
        embeddings = Dataloader.embed_texts(chuck)
        embeddings

[2025-11-07 04:55:56,869: INFO: common]: yaml file: config/config.yaml loaded successfully]
[2025-11-07 04:55:56,910: INFO: 2089868215]: data/pdfs/test.txt]
[2025-11-07 04:55:58,521: INFO: _client]: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:batchEmbedContents "HTTP/1.1 200 OK"]


In [24]:
print(embeddings)

[[-0.005617438, 0.008495868, 0.017205797, -0.08446541, 0.0055908402, 0.011333778, 0.0043527978, 0.0069937385, 0.00044334438, -0.0056814346, -0.013819317, -0.011128241, -0.011873111, -0.0035818026, 0.17536885, 0.00049435685, -0.0011655677, -0.011206717, -0.0058083637, -0.008149279, 0.014132847, -0.009215714, 0.019181667, -0.00720096, 0.011513603, -0.01975216, 0.0251996, -0.0030699696, 0.016812691, -0.0022829904, 0.0066344673, 0.007260402, 0.0065481043, 0.006995285, 0.0017379463, 0.022768488, 0.007883567, -0.009434932, -0.0021569643, 0.0074834567, 0.0033947416, -0.0030386408, -0.013111737, 0.005927797, 0.0017905635, 1.9291994e-05, 0.008862203, -0.029559094, 0.0012947869, 0.005554727, -0.008499042, -0.0077377935, 0.0020167166, -0.30317968, -0.0011326848, 0.014206104, -0.0024229574, 0.0017306405, -0.009350897, -2.1243948e-05, 0.00328641, 0.022914067, -0.000611865, -0.02466761, -0.0014612799, 0.0064293216, 0.0032771577, 0.007917133, -0.0066612456, -0.0041118073, -0.0029607632, 0.0053830403,